In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from IPython.display import display
import datetime
import re
import csv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic # great_circle


mypath = r"C:\Users\justr\Documents\dci-2024\raw data"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

with open('inputs/corps_data.csv', 'r') as f:
  list_of_corps = csv.reader(f)
  list_of_corps = [x[0] for x in list_of_corps]

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)

for f in files:
    print(f)

20240626rockford.htm
20240628muncie.htm
20240628walnut.htm
20240629madison.htm
20240629shelton.htm
20240629walnut.htm
20240630lisle.htm
20240630sanbernardino.htm
20240702mason.htm
20240703cedarburg.htm
20240705clovis.htm
20240705portland.htm
20240705rockford.htm
20240706dciwest.htm
20240706seattle.htm
20240706whitewater.htm
20240707lacrosse.htm
20240707sacramento.htm
20240708kennewick.htm
20240708sevierville.htm
20240709boise.htm
20240709mankato.htm
20240709newnan.htm
20240710ogden.htm
20240711dubuque.htm
20240711muscle.htm
20240712cheyenne.htm
20240712riverside.htm
20240713clifton.htm
20240713datr.htm
20240713dekalb.htm
20240713littlerock.htm
20240713vista.htm
20240714ankeny.htm
20240714cerritos.htm
20240715canyon.htm
20240715olathe.htm
20240716abeliene.htm
20240716brokenarrow.htm
20240718denton.htm
20240718waco.htm
20240719bellbrook.htm
20240719prairieview.htm
20240720brunswick.htm
20240720landisville.htm


In [2]:
scores_list = list()
corps_dict = dict()


geolocator = Nominatim(user_agent="hi")

# fix this!
scores_table = pd.DataFrame(np.zeros([1,18]))
scores_table.columns = ['Corps','Date','Location','Performance Slot','General Effect 1','General Effect 2','General Effect Total','Visual Proficiency','Visual Analysis','Color Guard','Visual Total','Music Brass','Music Analysis','Music Percussion','Music Total','Sub Total','Penalty','Total Score']


comps_data = list()

corps_classes = ['DCI World Class', 'DCI Open Class','DCI All Age World Class','DCI All Age Open Class','DCI All Age Class A']

for (show_number, file) in enumerate(files):
    # print(file)

    df_list = pd.read_html("raw data/"+file)
    df = df_list[-1]
    df = df.applymap(lambda x: re.sub('[^!-~]+',' ',x).strip() if type(x) == str else x)

    date_and_place_and_comp = df.iloc[0, 0]
    comp = None
    if len(date_and_place_and_comp.split(sep='--')) > 1:
        (date_and_place, comp) = date_and_place_and_comp.split(sep='--')
    else:
        date_and_place = date_and_place_and_comp
    chunks = date_and_place.split(sep=' ')
    place = " ".join(chunks[1:])


    # creating corps static file:
    corps_class = None
    running_name = ""
    for r in df.iloc[4:,0]: # this slice could be buggy, blind assumption that we can always skip the first 4 lines
        # print(f"r is {r}")
        if r in corps_classes:
            corps_class = r
        if corps_class != r:
            if running_name == "":
                running_name = str(r)
            else:
                if type(r) is str:
                    running_name += " " + r
                if running_name != 'nan':
                    corps_dict.update({running_name: corps_class})
                    running_name = ""
    
    for (i, r) in enumerate(df[0]):
        if type(r) == str:
            if r in corps_classes:
                df.drop(i, inplace=True)


    df = df.iloc[4:-1].reset_index(drop=True)


    # grab names
    names = df[0].reset_index(drop=True)

    clean_names = list()
    for (i, r) in enumerate(names):
        if (i % 2) == 0:
            running_name = str(r)
        else:
            try:
                float(r)
            except:
                running_name += " " + r
            clean_names.append(running_name)
    # print(clean_names)

    df = df.iloc[::2, :].reset_index(drop=True)
    df.columns = ['Corps','Performance Slot','General Effect 1','General Effect 2','General Effect Total','Visual Proficiency','Visual Analysis','Color Guard','Visual Total','Music Brass','Music Analysis','Music Percussion','Music Total','Sub Total','Penalty','Total Score']
    df['Corps'] = clean_names
    df.insert(1,column='Date',value=pd.to_datetime(chunks[0]))
    df.insert(2,column='Location',value=place)
    df.insert(0,column='Show Number',value=show_number+1)
    df.iloc[:,5:] = df.iloc[:,5:].astype(float)

    
    for (i, r) in df.iterrows():
        if np.isnan(r['Total Score']):
            df.drop(i, inplace=True)            
    
    df['Performance Slot'] = df['Performance Slot'].astype(int)
    # num_performances = df['Performance Slot'].max()

    # competition
    location = geolocator.geocode(place)
    df["Competition Latitude"] = location.latitude
    df["Competition Longitude"] = location.longitude

    scores_list.append(df)
    comps_data.append([place, comp])#, num_performances])

scores_table = pd.concat(scores_list)
scores_table.reset_index(inplace=True,drop=True)

scores_table['Days from Season Start'] = (scores_table['Date'] - scores_table['Date'].min()).apply(lambda x: x.days)

corps_table = pd.DataFrame(corps_dict.items(), columns=['Corps', 'Class'])
corps_table.set_index('Corps',inplace=True)

corps_rename_dict = {
    "Vanguard": "Santa Clara Vanguard",
    "Academy": "The Academy",
    "Cavaliers": "The Cavaliers",
    "Bushwackers": "Bushwackers Drum Corps",
    "Connecticut Hurricanes": "Hurricanes",
    "Battalion": "The Battalion"
}

for (k, v) in corps_rename_dict.items():
    corps_table.rename(index={k: v}, inplace=True)
    scores_table.replace(k, v, inplace=True)

shell_corps_table = pd.read_csv('inputs/corps_data.csv').set_index('Corps')
corps_table = shell_corps_table.join(other=corps_table,how='outer')

comps_table = pd.DataFrame(comps_data, columns=['Location', 'DCI Name'])#,'Number of Performances'])

for corps in corps_table.index:
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    if scores_table_corps.shape[0] == 0:
        continue
    scores_table_corps_last = scores_table_corps.iloc[-1,:].to_dict()
    for (k, v) in scores_table_corps_last.items():
        if k != "Corps":
            corps_table.loc[corps,k] = v
    # corps_table.loc[corps,"Average Performace Slot"] = scores_table_corps['Performance Slot'].mean() / comps_table.loc[]

corps_table['Letters in Name'] = [len(x.replace(' ','')) for x in list(corps_table.index)]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(corps_table)
    # display(scores_table)
    # display(comps_table)

,Home Location,Latitude,Longitude,Conference,Class,Show Number,Date,Location,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,Color Guard,Visual Total,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,Days from Season Start,Letters in Name
Corps,,,,,,,,,,,,,,,,,,,,,,,,,,,
7th Regiment,"New London, CT",41.355619,-72.099780,NaN,DCI Open Class,5.0,2024-06-29,"Shelton, Connecticut",1.0,11.1,11.2,22.3,8.8,12.4,10.9,16.05,8.9,11.1,10.0,15.00,53.35,0.0,53.35,41.320391,-73.083655,3.0,11
Atlanta CV,"Atlanta, GA",33.748992,-84.390264,NaN,DCI All Age World Class,23.0,2024-07-09,"Newnan, Georgia",1.0,15.5,15.0,30.5,15.8,15.3,15.2,23.15,16.2,15.4,14.4,23.00,76.65,0.0,76.65,33.380672,-84.799657,13.0,9
Blue Devils,"Concord, CA",37.976852,-122.033562,Pacific,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",7.0,18.2,18.4,36.6,18.0,18.3,18.3,27.30,18.2,18.0,18.3,27.25,91.15,0.0,91.15,30.085792,-95.990706,23.0,10
Blue Devils B,"Concord, CA",37.976852,-122.033562,NaN,DCI Open Class,35.0,2024-07-14,"Cerritos, California",5.0,14.9,14.7,29.6,15.5,14.7,14.4,22.30,14.5,14.7,14.4,21.80,73.70,0.0,73.70,33.864429,-118.053932,18.0,11
Blue Devils C,"Concord, CA",37.976852,-122.033562,NaN,DCI Open Class,18.0,2024-07-07,"Sacramento, California",2.0,11.0,10.4,21.4,9.3,11.3,10.0,15.30,10.2,10.7,10.2,15.55,52.25,0.0,52.25,38.581061,-121.493895,11.0,11
Blue Knights,"Denver, CO",39.739236,-104.984862,Southwestern,DCI World Class,41.0,2024-07-18,"Waco, Texas",7.0,15.8,15.8,31.6,16.0,16.0,15.3,23.65,16.0,16.0,15.5,23.75,79.00,0.0,79.00,31.549190,-97.147463,22.0,11
Blue Stars,"La Crosse, WI",43.812284,-91.251435,Midwestern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",1.0,17.0,16.5,33.5,16.8,16.7,17.0,25.25,16.7,16.6,16.0,24.65,83.40,0.0,83.40,30.085792,-95.990706,23.0,9
Bluecoats,"Canton, OH",40.798546,-81.374951,Eastern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",5.0,18.4,18.7,37.1,18.3,18.6,17.9,27.40,18.6,18.2,18.4,27.60,92.10,0.0,92.10,30.085792,-95.990706,23.0,9
Boston Crusaders,"Boston, MA",42.355433,-71.060511,Eastern,DCI World Class,40.0,2024-07-18,"Denton, Texas",6.0,17.8,18.3,36.1,17.5,17.3,18.1,26.45,17.9,17.8,18.5,27.10,89.65,0.0,89.65,33.183879,-97.141342,22.0,15


In [3]:
current_scores = dict()
latest_perf = dict()

for corps in set(scores_table['Corps']):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    current_scores.update({corps: list(scores_table_corps['Total Score'])[-1]})
    latest_perf.update({corps: list(scores_table_corps['Date'])[-1]})

corps_table = corps_table.join(pd.DataFrame(current_scores.items(),columns=['Corps','Latest Score']).set_index('Corps'),how='outer')
corps_table = corps_table.join(pd.DataFrame(latest_perf.items(),columns=['Corps','Last Performance Date']).set_index('Corps'),how='outer')
corps_table.sort_values(by=['Latest Score','Corps'],inplace=True,ascending=[False, True])
display(corps_table)

,Home Location,Latitude,Longitude,Conference,Class,Show Number,Date,Location,Performance Slot,General Effect 1,...,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,Days from Season Start,Letters in Name,Latest Score,Last Performance Date
Corps,,,,,,,,,,,,,,,,,,,,,
Bluecoats,"Canton, OH",40.798546,-81.374951,Eastern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",5.0,18.4,...,27.60,92.10,0.0,92.10,30.085792,-95.990706,23.0,9,92.10,2024-07-19
Blue Devils,"Concord, CA",37.976852,-122.033562,Pacific,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",7.0,18.2,...,27.25,91.15,0.0,91.15,30.085792,-95.990706,23.0,10,91.15,2024-07-19
Carolina Crown,"Fort Mill, SC",35.007370,-80.945076,Eastern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",6.0,17.9,...,27.05,89.80,0.0,89.80,30.085792,-95.990706,23.0,13,89.80,2024-07-19
Boston Crusaders,"Boston, MA",42.355433,-71.060511,Eastern,DCI World Class,40.0,2024-07-18,"Denton, Texas",6.0,17.8,...,27.10,89.65,0.0,89.65,33.183879,-97.141342,22.0,15,89.65,2024-07-18
Reading Buccaneers,"Reading, PA",40.335345,-75.927949,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",3.0,17.8,...,26.30,87.95,0.0,87.95,40.095120,-76.410196,24.0,17,87.95,2024-07-20
Phantom Regiment,"Rockford, IL",42.271394,-89.093966,Midwestern,DCI World Class,40.0,2024-07-18,"Denton, Texas",5.0,17.4,...,25.75,86.60,0.0,86.60,33.183879,-97.141342,22.0,15,86.60,2024-07-18
Mandarins,"Sacramento, CA",38.581061,-121.493895,Pacific,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",4.0,17.5,...,25.55,86.40,0.0,86.40,30.085792,-95.990706,23.0,9,86.40,2024-07-19
Bushwackers Drum Corps,"Princeton, NJ",40.349695,-74.659738,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",2.0,17.3,...,25.35,85.30,0.0,85.30,40.095120,-76.410196,24.0,20,85.30,2024-07-20
Hawthorne Caballeros,"Hawthorne, NJ",40.949265,-74.153755,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",1.0,17.1,...,25.75,85.05,0.0,85.05,40.095120,-76.410196,24.0,19,85.05,2024-07-20


In [4]:
today = datetime.date.today()
day_of_season = (pd.Timestamp(today) - scores_table['Date'].min()).days

In [11]:
scores_table_diff = scores_table.copy(deep=True)
scores_table_diff.iloc[:,5:] = 0.0
scores_table_diff

,Show Number,Corps,Date,Location,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,...,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,Days from Season Start
0,1,Boston Crusaders,2024-06-26,"Rockford, Michigan",5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,Phantom Regiment,2024-06-26,"Rockford, Michigan",2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1,The Cavaliers,2024-06-26,"Rockford, Michigan",4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1,Blue Stars,2024-06-26,"Rockford, Michigan",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1,Colts,2024-06-26,"Rockford, Michigan",3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,45,Bushwackers Drum Corps,2024-07-20,"Landisville, Pennsylvania",2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
284,45,Hawthorne Caballeros,2024-07-20,"Landisville, Pennsylvania",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
285,45,White Sabers,2024-07-20,"Landisville, Pennsylvania",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
286,45,Fusion Core,2024-07-20,"Landisville, Pennsylvania",2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [9]:
# determine difference in score by comp

growth_fit_xab = lambda x,a,b: (a * x) + b

scores_table_diff = scores_table.copy(deep=True)
scores_table_diff.iloc[:,5:] = 0.0
scores_table_rate = scores_table.copy(deep=True)
scores_table_rate.iloc[:,5:] = 0.0

for corps in set(scores_table['Corps']):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    scores_table_diff_corps = scores_table_diff.loc[scores_table_diff['Corps'] == corps]
    scores_table_rate_corps = scores_table_rate.loc[scores_table_rate['Corps'] == corps]
    
    for c in scores_table_corps.columns:
        if c not in ['Corps','Location']:
            scores_table_diff_corps.loc[:,c] = scores_table_corps.loc[:,c].diff()
    
    for c in scores_table_corps.columns:
        if c not in ['Corps','Location']:
            scores_table_rate_corps.loc[:,c] = scores_table_diff_corps.loc[:,c] / scores_table_diff_corps.loc[:,"Date"].apply(lambda x: x.days)

    for (i, r) in scores_table_diff_corps.iterrows():
        scores_table_diff.iloc[i,:] = r
    
    for (i, r) in scores_table_rate_corps.iterrows():
        scores_table_rate.iloc[i,:] = r
    
    # WIP CODE RIGHT HERE
    # display(scores_table_corps)
    scores_table_corps_shifted = scores_table_corps.shift(1)
    # display(scores_table_corps_shifted)

    for (i, r) in scores_table_corps_shifted.iterrows():
        if scores_table_corps_shifted.index[0] != i:
            geodesic_distance = geodesic((scores_table_corps.loc[i, "Competition Latitude"], scores_table_corps.loc[i, "Competition Longitude"]), (r["Competition Latitude"], r["Competition Longitude"])).mi
            scores_table_diff.loc[i, "Geodesic Distance"] = geodesic_distance
    
    corps_table.loc[corps, "Average Geodesic Distance"] = scores_table_diff[scores_table_diff['Corps'] == corps]["Geodesic Distance"].mean()
    corps_table.loc[corps, "Total Geodesic Distance"] = scores_table_diff[scores_table_diff['Corps'] == corps]["Geodesic Distance"].sum()
    corps_table.loc[corps, "Geodesic Distance Per Day"] = corps_table.loc[corps, "Total Geodesic Distance"] / scores_table_corps.shape[0]

    # display(scores_table_diff_corps)

    if int(scores_table_corps.shape[0]) > 2: # must have more than 2 data points lowkey
        ((a, b), cov) = np.polyfit(scores_table_corps['Days from Season Start'].astype(int), scores_table_corps['Total Score'].astype(float), 1, cov=True)
        (sa, sb) = np.sqrt(np.diag(cov))
        corps_table.loc[corps, "RMSE"] = (scores_table_corps['Total Score'] - scores_table_corps['Days from Season Start'].apply(lambda x: growth_fit_xab(x, a, b))).std()

        for i in range(1,7+1):
            est = growth_fit_xab(day_of_season + i, a + (sa * 0), b + (sb * 0))
            upper = growth_fit_xab(day_of_season + i, a + (sa * i), b + (sb * i))
            lower = growth_fit_xab(day_of_season + i, a + (sa * -i), b + (sb * -i))
            est = 100.0 if est > 100.0 else est
            upper = 100.0 if upper > 100.0 else upper
            lower = 100.0 if lower > 100.0 else lower
            corps_table.loc[corps, f"D+{i}"] = 0.0 if est < 0.0 else est
            corps_table.loc[corps, f"D+{i} U"] = 0.0 if upper < 0.0 else upper
            corps_table.loc[corps, f"D+{i} L"] = 0.0 if lower < 0.0 else lower
    
    corps_table.loc[corps,'Number of Shows'] = int(scores_table_corps.shape[0])
    corps_table.loc[corps, 'Average Rest'] = scores_table_diff_corps.loc[:,'Date'].apply(lambda x: x.days).mean()
    corps_table.loc[corps, 'Average Score Improvement'] = scores_table_rate_corps.loc[:,'Total Score'].mean()
    
    # normal ranges from 0 to 1
    corps_table.loc[corps,'Average of Last 3 Scores'] = scores_table_corps['Total Score'].iloc[-3:].mean()
    corps_table.loc[corps,'Average Normal General Effect'] = scores_table_corps['General Effect Total'].iloc[-3:].mean() / 40
    corps_table.loc[corps,'Average Normal Visual'] = scores_table_corps['Visual Total'].iloc[-3:].mean() / 30
    corps_table.loc[corps,'Average Normal Music'] = scores_table_corps['Music Total'].iloc[-3:].mean() / 30
    corps_table.loc[corps,'Average Normal Brass'] = scores_table_corps['Music Brass'].iloc[-3:].mean() / 20
    corps_table.loc[corps,'Average Normal Percussion'] = scores_table_corps['Music Percussion'].iloc[-3:].mean() / 20

    if len(set(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)) < len(list(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)):
        corps_table.loc[corps, 'Best Caption'] = 'Multiple'
    else:
        max_column_index = np.argmax(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)
        corps_table.loc[corps, 'Best Caption'] = ['General Effect', 'Visual', 'Music'][max_column_index]


    corps_table.loc[corps, 'Best Music'] = ['Brass', 'Percussion'][int(corps_table.loc[corps, 'Average Normal Brass'] < corps_table.loc[corps, 'Average Normal Percussion'])] if corps_table.loc[corps, 'Average Normal Brass'] != corps_table.loc[corps, 'Average Normal Percussion'] else 'Neither'

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     if corps == "Phantom Regiment":
    #         display(scores_table_corps)
    #         display(scores_table_diff_corps)
    #         display(scores_table_rate_corps)

corps_table['Rank of Last 3 Scores'] = corps_table['Average of Last 3 Scores'].rank(ascending=False, method='max')
corps_table['Rank of Last Score'] = corps_table['Latest Score'].rank(ascending=False, method='max')
corps_table['Rank of Last 3 Scores No All-Age'] = corps_table.loc[['All Age' not in str(x) for x in corps_table['Class']], 'Average of Last 3 Scores'].rank(ascending=False, method='max')
corps_table['Rank of Last Score No All-Age'] = corps_table.loc[['All Age' not in str(x) for x in corps_table['Class']], 'Latest Score'].rank(ascending=False, method='max')

def ordinal(n):
    if str(n) == 'nan':
        return None
    else:
        return "%d%s" % (int(n),"tsnrhtdd"[(int(n)//10%10!=1)*(int(n)%10<4)*int(n)%10::4])
    
corps_table['Ordinal Rank of Last 3 Scores'] = corps_table['Rank of Last 3 Scores'].apply(ordinal)
corps_table['Ordinal Rank of Last Score'] = corps_table['Rank of Last Score'].apply(ordinal)
corps_table['Ordinal Rank of Last 3 Scores No All-Age'] = corps_table['Rank of Last 3 Scores No All-Age'].apply(ordinal)
corps_table['Ordinal Rank of Last Score No All-Age'] = corps_table['Rank of Last Score No All-Age'].apply(ordinal)


championship_rounds_table = pd.DataFrame([None,'Prelims','Semis','Finals'],columns=['Championship Rounds'])

def calc_championship_round(rank: float):
    if rank <= 12:
        return championship_rounds_table['Championship Rounds'][3]
    elif rank <= 25:
        return championship_rounds_table['Championship Rounds'][2]
    elif rank <= 40:
        return championship_rounds_table['Championship Rounds'][1]
    else:
        return championship_rounds_table['Championship Rounds'][0]

corps_table['Projected Championship Round'] = corps_table['Rank of Last 3 Scores No All-Age'].apply(calc_championship_round)

captions_table = pd.DataFrame(['General Effect','Visual','Music','Multiple','Brass','Percussion','Neither'],columns=['Caption Title'])

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(corps_table)
    # display(scores_table_diff)

In [51]:
# calculate placements per comp per group

# scores_table = scores_table.copy(deep=True)
set_show_numbers = set(scores_table['Show Number'])
dict_corps_table_class = corps_table['Class'].to_dict()

for show_number in set_show_numbers:
    scores_table_show = scores_table[scores_table['Show Number'] == show_number].copy(deep=True)
    scores_table_show.loc[:,'Class'] = scores_table_show['Corps'].map(dict_corps_table_class)

    for performance_class in set(scores_table_show['Class']):
        scores_table_show_class = scores_table_show[scores_table_show['Class'] == performance_class].copy(deep=True)
        scores_table_show_class.sort_values('Total Score', ascending=False, inplace=True)
        
        scores_table_show_class['Show and Class Placement'] = scores_table_show_class['Total Score'].rank(ascending=False, method='min')
        set_placement = scores_table_show_class['Show and Class Placement'].astype(int).to_dict()

        scores_table_show_class['Show and Class Points'] = scores_table_show_class['Total Score'].rank(ascending=True, method='max')
        set_points = scores_table_show_class['Show and Class Points'].astype(int).to_dict()

        for (k, v) in set_placement.items():
            scores_table.loc[k,'Show and Class Placement'] = v

        for (k, v) in set_points.items():
            scores_table.loc[k,'Show and Class Points'] = v


scores_table['Show and Class Placement'] = scores_table['Show and Class Placement'].astype(int)
scores_table['Show and Class Points'] = scores_table['Show and Class Points'].astype(int)

max_placement = scores_table['Show and Class Placement'].max()

corps_table['Participate Place'] = 0
for (corps, row) in corps_table.iterrows():
    for i in range(1, max_placement + 1):
        corps_table.loc[corps, f'{i} Place'] = scores_table['Show and Class Placement'][scores_table['Show and Class Placement'] == i][scores_table['Corps'] == corps].count()
        
    corps_table.loc[corps, 'Participate Place'] = corps_table.loc[corps, [f'{i} Place' for i in range(3 + 1, max_placement + 1)]].sum()

    corps_table.loc[corps, 'Total Points'] = scores_table['Show and Class Points'][scores_table['Corps'] == corps].sum()

corps_table['Total Points'] = corps_table['Total Points'].astype(int)

for i in range(1, max_placement + 1):
    corps_table[f'{i} Place'] = corps_table[f'{i} Place'].astype(int)

corps_table_sorted_placement = corps_table.sort_values(by=[f'{i} Place' for i in range(1, max_placement + 1)], ascending=[False for _ in range(1, max_placement + 1)]).reset_index().copy(deep=True)
corps_table_sorted_placement['Placement Rank'] = corps_table_sorted_placement.index + 1

dict_placement = corps_table_sorted_placement.set_index("Corps")['Placement Rank'].to_dict()

for (k, v) in dict_placement.items():
    corps_table.loc[k, 'Placement Rank'] = v
display(scores_table[scores_table['Corps'] == 'Phantom Regiment'])
display(corps_table)

,Show Number,Corps,Date,Location,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,...,Music Percussion,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,Days from Season Start,Show and Class Placement,Show and Class Points
1,1,Phantom Regiment,2024-06-26,"Rockford, Michigan",2,14.8,14.2,29.0,14.0,13.6,...,13.1,20.55,70.15,0.0,70.15,43.119721,-85.559604,0,2,4
7,2,Phantom Regiment,2024-06-28,"Muncie, Indiana",3,14.5,14.7,29.2,14.5,14.2,...,14.3,21.7,72.15,0.0,72.15,40.193689,-85.386527,2,3,4
44,7,Phantom Regiment,2024-06-30,"Lisle, Illinois",4,15.0,14.7,29.7,15.0,14.5,...,15.0,22.05,73.8,0.0,73.8,41.801159,-88.074769,4,3,5
62,9,Phantom Regiment,2024-07-02,"Mason, OH",1,15.6,15.0,30.6,15.2,14.7,...,15.0,22.45,75.6,0.0,75.6,39.360059,-84.309939,6,4,1
77,13,Phantom Regiment,2024-07-05,"Rockford, Illinois",4,15.7,15.3,31.0,15.6,15.6,...,15.8,23.2,77.5,0.0,77.5,42.271394,-89.093966,9,1,4
99,16,Phantom Regiment,2024-07-06,"Whitewater, Wisconsin",2,15.9,15.8,31.7,15.8,15.9,...,16.0,23.7,79.15,0.0,79.15,42.833642,-88.729268,10,2,4
106,17,Phantom Regiment,2024-07-07,"LaCrosse, Wisconsin",2,15.8,16.9,32.7,16.0,16.1,...,16.0,23.85,80.65,0.0,80.65,43.904302,-91.104106,11,2,4
134,22,Phantom Regiment,2024-07-09,"Mankato, Minnesota",3,16.3,16.4,32.7,16.1,16.6,...,16.1,24.1,81.4,0.0,81.4,44.163466,-93.999351,13,1,3
155,25,Phantom Regiment,2024-07-11,"Dubuque, Iowa",3,16.4,16.6,33.0,16.5,16.7,...,16.8,24.9,82.9,0.0,82.9,42.500624,-90.664799,15,1,4
195,31,Phantom Regiment,2024-07-13,"DeKalb, Illinois",6,16.8,17.0,33.8,16.9,16.9,...,17.2,25.65,84.75,0.0,84.75,41.890345,-88.771395,17,2,5


,Home Location,Latitude,Longitude,Conference,Class,Show Number,Date,Location,Performance Slot,General Effect 1,...,2 Place,3 Place,4 Place,5 Place,6 Place,7 Place,8 Place,9 Place,Total Points,Placement Rank
Corps,,,,,,,,,,,,,,,,,,,,,
Bluecoats,"Canton, OH",40.798546,-81.374951,Eastern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",5.0,18.4,...,0,0,0,0,0,0,0,0,45,2.0
Blue Devils,"Concord, CA",37.976852,-122.033562,Pacific,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",7.0,18.2,...,2,0,0,0,0,0,0,0,56,1.0
Carolina Crown,"Fort Mill, SC",35.007370,-80.945076,Eastern,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",6.0,17.9,...,1,3,1,0,0,0,0,0,38,11.0
Boston Crusaders,"Boston, MA",42.355433,-71.060511,Eastern,DCI World Class,40.0,2024-07-18,"Denton, Texas",6.0,17.8,...,5,0,0,0,0,0,0,0,45,7.0
Reading Buccaneers,"Reading, PA",40.335345,-75.927949,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",3.0,17.8,...,0,0,0,0,0,0,0,0,11,12.0
Phantom Regiment,"Rockford, IL",42.271394,-89.093966,Midwestern,DCI World Class,40.0,2024-07-18,"Denton, Texas",5.0,17.4,...,4,2,2,1,0,0,0,0,50,8.0
Mandarins,"Sacramento, CA",38.581061,-121.493895,Pacific,DCI World Class,43.0,2024-07-19,"Prairie View, Texas",4.0,17.5,...,4,4,1,0,1,0,0,0,42,20.0
Bushwackers Drum Corps,"Princeton, NJ",40.349695,-74.659738,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",2.0,17.3,...,3,0,0,0,0,0,0,0,8,34.0
Hawthorne Caballeros,"Hawthorne, NJ",40.949265,-74.153755,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",1.0,17.1,...,0,3,0,0,0,0,0,0,5,41.0


In [52]:
from pybaseball import standings

# mlb_table = pd.read_csv('mlb/mlb.csv')
# display(mlb_table)

data = standings()

mlb_table = pd.concat(data, axis=0).sort_values('W-L%',ascending=False).reset_index(drop=True)
mlb_table.index = mlb_table.index + 1

display(mlb_table)

,Tm,W,L,W-L%,GB
1,Philadelphia Phillies,63,36,.636,--
2,Baltimore Orioles,60,39,.606,--
3,Cleveland Guardians,59,39,.602,--
4,Los Angeles Dodgers,59,41,.590,--
5,New York Yankees,59,42,.584,2.0
6,Milwaukee Brewers,57,42,.576,--
7,Atlanta Braves,54,44,.551,8.5
8,Minnesota Twins,54,44,.551,5.0
9,Kansas City Royals,55,45,.550,5.0
10,Boston Red Sox,53,45,.541,6.5


In [53]:
import plotly.graph_objects as go
import plotly.colors as pc

(a, b) = np.polyfit(scores_table['Days from Season Start'].astype(int), scores_table['Total Score'].astype(float), 1)
growth_fit = lambda x: (a * x) + b

scores_table['Total Score Residual'] = scores_table['Total Score'] - scores_table['Days from Season Start'].apply(growth_fit)

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, corps) in enumerate(corps_table.index):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    scores_table_diff_corps = scores_table_diff.loc[scores_table['Corps'] == corps]
    scores_table_rate_corps = scores_table_rate.loc[scores_table['Corps'] == corps]
    fig.add_trace(go.Scatter(
        x=scores_table_corps['Date'],
        y=scores_table_corps['Total Score'],
        name=f'#{ranking} ({corps_table.loc[corps,"Latest Score"]:.3f}) {corps}',
        mode='lines+markers',
        connectgaps=True,
        line=dict(
            #shape='hv',
            color=trace_colors[i % len(trace_colors)]
        ),
        text=[
            f'<b>{x[0]}</b><br><br>' + 
            f"{x[1].strftime('%A, %d %B %Y')}<br>"
            f"{x[2]}<br><br>" +
            f"Competition Score: {x[3]:.3f}<br>"
            f"Difference: {x[4]:+.3f}<br>"
            f"Score Growth: {x[5]:+.3f} per day"
            for x in zip(
                scores_table_corps['Corps'],
                scores_table_corps['Date'],
                scores_table_corps['Location'],
                scores_table_corps['Total Score'],
                scores_table_diff_corps['Total Score'],
                scores_table_rate_corps['Total Score']
            )
        ]
    ))
    ranking += 1

updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'

def active_ranking(cat):
    i = 0
    t = 0
    retList = list()
    for c in cat:
        retList.append(f"#{i+1} ({corps_table['Latest Score'][t]:.3f}) {corps_table.index[t]}")
        if c is True:
            i += 1
        t += 1
    return retList 

dropdown_labels = ["All Corps"] + corps_classes

dropdown_categories = [[True for _ in corps_table['Class']]] + [
    [x == corps_classes[i] for x in corps_table['Class']] for i in range(len(corps_classes))
]

dropdown_dicts = [
    dict(
    label=label,
    method='restyle',
    args=[{"visible": category, "name": active_ranking(category)}]
    ) for (label, category) in zip(dropdown_labels, dropdown_categories)
]


fig.update_layout(
    title=f'<b>DCI 2024 Scores by Roman Ramirez</b><br>{updated_time}<br>',
    xaxis_title='<b>Date</b>',
    yaxis_title='<b>Total Score</b>',
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_dicts
            ,
        )       
    ]
)

customdata = np.stack((
    list(scores_table['Location']),
    list(scores_table_diff['Total Score']),
    list(scores_table['Corps'])
    ), axis=0)
hovertemplate = (
    '%{text}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Latest Score</i>) <i>Corps</i></b>'
)

fig.show()
fig.write_html("index.html")

export(scores_table, 'scores', True)
export(scores_table_diff, 'scores_diff', True)
export(scores_table_rate, 'scores_rate', True)
export(corps_table, 'corps', True)
export(comps_table, 'comps', True)
export(captions_table, 'captions', True)
export(championship_rounds_table, 'championship_rounds', True)
export(mlb_table, 'mlb', True)

#initialze the excel writer
writer = pd.ExcelWriter('exports/dci_data.xlsx', engine='xlsxwriter')

#store your dataframes in a  dict, where the key is the sheet name you want
frames = {
    'scores': scores_table, 
    'scores_diff': scores_table_diff,
    'scores_rate': scores_table_rate,
    'corps': corps_table, 
    'comps': comps_table, 
    'captions': captions_table, 
    'championship rounds': championship_rounds_table, 
    'mlb': mlb_table
    }

#now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): # .use .items for python 3.X
    frame.to_excel(writer, sheet_name = sheet)

#critical last step
writer.close()

In [54]:
#to dos

# in import, create "static" corps master file
# # contains class, and other corps-related info

# SQL join this on scores table
# add buttons to filter by class